Link to github: https://github.com/albert-moller/CSSAssignment1.git

Group members: Albert Frisch Møller (s214610) and Mark Andrawes (s214654)

For this assignment each group member contributed equally to every aspect of the assignment

### Assignment 1

#### Part 1: Web-Scraping

In [1]:
#Importing necessary Web Scraping and Data Storage packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import networkx as nx
import json

if not os.path.exists("./data"):
    os.mkdir("./data")

In [2]:
LINK = "https://ic2s2-2023.org/program"
r = requests.get(LINK)
soup = BeautifulSoup(r.content)
researchers = []

table_rows = soup.find_all("tr")
for row in table_rows:
    for tag in row.find_all("a"):
        if re.search("Keynote", tag.text):
            string = tag.text
            string = string.replace("Keynote - ", "")
            string = string.strip()
            researchers.append(string)

plenaries = soup.find_all(class_='nav_list')

for plenary in plenaries:
    italic_names = plenary.find_all('i')
    italic_names = [i.get_text() for i in italic_names]

    for entry in italic_names:
        speakers = entry.split(',')
        for speaker in speakers:
            name = speaker.strip()
            if name not in researchers:
                researchers.append(speaker)

researchers_df = pd.DataFrame(researchers, columns=['Full Name'])
researchers_df.to_csv('data/ics2_researchers.csv', index=False)

print(f"The number of unique researchers is {len(researchers_df)}")

The number of unique researchers is 1856


#### How many unique researchers do you get?

We obtained that there are 1856 unique researchers present at the International Conference in Computational Social Science for 2023

#### Explain the process you followed to web-scrape the page. Which choices did you make to accurately retreive as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices. (answer in max 150 words)

To webscrape the IC2S2 2023 program page, we inspected the HTML structure to identify the elements containing researcher names. Using BeautifulSoup, we collected the table rows ('tr') to extract the names of the keynote speakers. Additionally, we collected each 'i' tag within each plenary (found in the class 'nav_list') which contained multiple names of participants - we split these names and appended each one to the list. Finally, we stored the data in a Pandas DataFrame and saved as csv.


To retrieve the names accurately, we removed the 'Keynote -' prefix to cleanly retrieve the names of the keynote speakers. We used the '.strip()' function to remove any leading/trailing whitespace. This was done to ensure that we can accurately compare names when checking for duplicates. We assessed the quality by checking for duplicates by only adding new names to the list. This ensured the list only included unique participants.



#### Part 2: Ready Made vs Custom Made Data

##### What are pros and cons of the custom-made data used in Centola's experiment (the first study presented in the lecture) and the ready-made data used in Nicolaides's study (the second study presented in the lecture)? You can support your arguments based on the content of the lecture and the information you read in Chapter 2.3 of the book (answer in max 150 words)

In Centola's experiment, custom-made data was collected specifically for the study, allowing for full control over the environment and the variables. As explained in “Bit by Bit”, this enhances the relevance and accuracy of the findings to the research question, which is a significant advantage. Additionally, this reduces ethical concerns in the data. On the other hand, custom-made data requires significant resources and time to create, which is a disadvantage.


In Nicolaides's study, ready-made data was used. As described in “Bit by Bit”, an advantage of this is the free accessibility of the data and its broader context. However, this type of data usually contains biases and may require adjustments in the methodology to align with the available data. Moreover, there may be issues with the quality and completeness of the data.


##### How do you think these differences can influence the interpretation of the results in each study? (answer in max 150 words)

The differences between custom-made and ready-made data can significantly influence the interpretation of results in the studies. For Centola's experiment, the control over the experimental conditions allowed for clearer ways of linking effects to specific causes, leading to these insights being more reliable. However, this control also means that the results might be less generalizable to real-world settings, as the controlled environment may not capture all realistic external factors. 

On the other hand, Nicolaides's use of ready-made data derived from existing datasets offers insights that are more reflective of real-world behaviors. This enhances the generalizability of the findings but makes it challenging to determine links between effects and causes due to potential confounders and biases in the data.
Hence, while custom-made data provides cleaner, more controlled insights, ready-made data offers broader perspectives on natural behaviors. These differences mean that we must carefully interpret results based on the data type.


#### Part 3: Gathering Research Articles using the OpenAlex API

In [4]:
#Step 1) Obtain ISC2 Research OpenAlex IDs (using the "authors" endpoint)

researchers = pd.read_csv("data/ics2_researchers.csv")
openalexids_df = pd.DataFrame(columns = ['id', 'display_name', 'works_api_url', 'h_index', 'works_count', 'country_code', 'cited_by_count'])

url = "https://api.openalex.org/authors"
index = 0

for researcher in tqdm(researchers["Full Name"], desc="Obtaining authors dataset"):
    params = {'search': researcher}
    response = requests.get(url, params=params)
    data = response.json()

    if not data['results']:
        continue

    author_data = data['results']
    index += 1
    if len(author_data) > 1:
        author_data = max(author_data, key=lambda x: x.get('relevance_score', 0))
    else:
        author_data = data['results'][0]

    try:
        id = author_data['id']
        display_name = author_data['display_name']
        works_api_url = author_data['works_api_url']
        h_index = author_data['summary_stats']['h_index']
        works_count = author_data['works_count']
        country_code = author_data['last_known_institution']['country_code']
        cited_by_count = author_data['cited_by_count'] 
    except:
        continue

    df_index = len(openalexids_df)
    openalexids_df.loc[df_index] = [id, display_name, works_api_url, h_index, works_count, country_code, cited_by_count]

    if index % 30 == 0:
        openalexids_df.to_csv("data/ics2_authors.csv", index=False)    

Obtaining authors dataset: 100%|██████████| 1856/1856 [20:38<00:00,  1.50it/s]


In [5]:
#Step 2) Use the "concepts" endpoint to obtain the concepts IDs for Sociology, Psychology, Economics, Political Science, Mathematics, Physics, Computer Science

fields = ["Sociology", "Psychology", "Economics", "Political Science", "Mathematics", "Physics", "Computer Science"]
concepts_id = {}

for field in fields:
    url = f"https://api.openalex.org/concepts?search={field}"
    response = requests.get(url)
    data = response.json()
    if data['results']:
        concept_id = data['results'][0]['id']
        _, concept_id = os.path.split(concept_id)
        concepts_id[field] = concept_id

quantitative_disciplines_filter = f"{concepts_id['Mathematics']}|{concepts_id['Physics']}|{concepts_id['Computer Science']}"
css_filter = f"{concepts_id['Sociology']}|{concepts_id['Psychology']}|{concepts_id['Economics']}|{concepts_id['Political Science']}"

print(f"Quantiative Disciplines Filter: {quantitative_disciplines_filter}")
print(f"Computational Social Science Filter: {css_filter}")

Quantiative Disciplines Filter: C33923547|C121332964|C41008148
Computational Social Science Filter: C144024400|C15744967|C162324750|C17744445


In [6]:
#Step 3) Use the "works" endpoint to obtain all the research articles authored by ICS2 participants

def fetch_author_works(author_id, social_sciences_ids, quantitative_disciplines_ids):
    _, openalex_id = os.path.split(author_id)
    filters = (
        f'author.id:{openalex_id}|{openalex_id}',
        f'cited_by_count:>10',
        f'authors_count:<10',
        f'concepts.id:({social_sciences_ids})',
        f'concepts.id:({quantitative_disciplines_ids})'
    )
    url = f"https://api.openalex.org/works?filter={','.join(filters)}per-page=200"

    response = requests.get(url)
    data = response.json()
    return data['results']

def process_author_works(author_id):
    author_works = fetch_author_works(author_id, css_filter, quantitative_disciplines_filter)
    papers_data = []
    abstracts_data = []
    for work in author_works:
        work_id = work['id']
        publication_year = work['publication_year']
        cited_by_count = work['cited_by_count']
        title = work['title']
        abstract_inverted_index = work.get('abstract_inverted_index', {})
        author_ids = [author['author']['id'] for author in work['authorships']]
        
        papers_data.append({
            'id': work_id,
            'publication_year': publication_year,
            'cited_by_count': cited_by_count,
            'author_ids': author_ids,
        })
        
        abstracts_data.append({
            'id': work_id,
            'title': title,
            'abstract_inverted_index': abstract_inverted_index,
        })
    return papers_data, abstracts_data
        
    
ICS2_authors = pd.read_csv("data/ics2_authors.csv")
ICS2_authors_filtered = ICS2_authors[(ICS2_authors['works_count'] >= 5) & (ICS2_authors['works_count'] <= 5000)]
author_ids = ICS2_authors_filtered['id'].tolist()

all_papers_data = []
all_abstracts_data = []

results = Parallel(n_jobs=4, backend="threading")(delayed(process_author_works)(author_id) for author_id in tqdm(author_ids))

for papers_data, abstracts_data in results:
    all_papers_data.extend(papers_data)
    all_abstracts_data.extend(abstracts_data)

papers_df = pd.DataFrame(all_papers_data, columns=["id", "publication_year", "cited_by_count", "author_ids"])
abstracts_df = pd.DataFrame(all_abstracts_data, columns=["id", "title", "abstract_inverted_index"])
papers_df.to_csv('data/papers.csv', index=False)
abstracts_df.to_csv('data/abstracts.csv', index=False)

100%|██████████| 1494/1494 [04:40<00:00,  5.32it/s]


In [7]:
# Step 4) Determine how many unique researchers have co-authored these works

co_authors = papers_df['author_ids']
unique_co_authors = []

for row in co_authors:
    for author in row:

        if author not in unique_co_authors:
            unique_co_authors.append(author)

print(f"Number of works listed in the ICSS2 papers dataframe: {len(papers_df)}")
print(f"Number of unique researchers that have co-authored the found works: {len(unique_co_authors)}")

Number of works listed in the ICSS2 papers dataframe: 12911
Number of unique researchers that have co-authored the found works: 13040


##### How many works are listed in your IC2S2 papers dataframe? How many unique researchers have co-authored these works?

There are 12911 works listed in the ICS2S2 papers dataframe. Additionally, there are 13040 unique researchers that have co-authored the found works.  

#### Describe the strategies you implemented to make your code more efficient. How did your approach affect your code's execution time? (answer in max 150 words)

To enhance code efficiency, our approach included batching requests to the OpenAlex API to query multiple authors simultaneously and applying filters directly within API requests to minimize data processing. We used Joblib's Parallel function which enabled concurrent processing, significantly reducing the total execution time. By adjusting the API's per-page limit to 200 works and implementing pagination, the script efficiently handled large datasets. These strategies ensured that the data retrieval process was both fast and resource-efficient, and the executing time was massively reduced by implementing these strategies compared to the case where they were not implemented.  

#### Reflect on the rationale behind setting specific thresholds for the total number of works by an author, the citation count, the number of authors per work, and the relevance of works to specific fields. How do these filtering criteria contribute to the relevance of the dataset you compiled? Do you believe any aspects of Computational Social Science research might be underrepresented or overrepresented as a result of these choices? (answer in max 150 words)

The purpose of the specified filtering criteria is to enhance the dataset’s relevance and manageability. Setting thresholds for an author’s total work count ensured that we only included those with a significant contribution to the field of Computational Social Science, while the citation count filter ensured that only works with greater impact were included. Limiting works authored by fewer than 10 individuals helped focus on collaborations that are more typical in Computational Social Science, and avoided overly large teams that might dilute the focus of the paper. Including only works that are relevant to Computational Social Science and a quantitative discipline ensured the dataset’s relevance to computational methodologies. These filters may lead to an underrepresentation of emerging research with fewer citations. On the other hand, well-cited collaborative research within the traditional Computational Social Science disciplines may be overrepresented. 

##### Part 4: The Network of Computational Social Scientists

In [8]:
# Network Construction

weighted_edgelist = []

for author_ids in tqdm(papers_df['author_ids'], desc="Computing weighted edgelist"):
    for i in range(len(author_ids)):
        for j in range(i+1, len(author_ids)):
            author_i = author_ids[i]
            author_j = author_ids[j]
            edge_exists = next(((index, edge) for index, edge in enumerate(weighted_edgelist) if (edge[0] == author_i and edge[1] == author_j) or (edge[0] == author_j and edge[1] == author_i)), None)
            if edge_exists == None:
                weighted_edgelist.append((author_i, author_j, 1))
            else:
                edge_index = edge_exists[0]
                weighted_edgelist[edge_index] = (edge_exists[1][0], edge_exists[1][1], edge_exists[1][2]+1)


Computing weighted edgelist: 100%|██████████| 12911/12911 [01:18<00:00, 164.13it/s]


In [9]:
# Graph Construction

G = nx.Graph()
G.add_weighted_edges_from(weighted_edgelist)

In [10]:
# Node Attributes

for index, data in ICS2_authors.iterrows():
    author_id = str(data['id'])
    display_name = data['display_name']
    country = data['country_code']
    citation_count = data['cited_by_count']
    year_of_first_publication = first_year = papers_df[papers_df['author_ids'].apply(lambda x: author_id in x)]['publication_year'].min()

    if author_id in G.nodes:
        G.nodes[author_id]['display_name'] = display_name
        G.nodes[author_id]['country'] = country
        G.nodes[author_id]['citation_count'] = citation_count
        G.nodes[author_id]['year_of_first_publication'] = int(year_of_first_publication)

data = nx.node_link_data(G)
with open('data/social_scientists_network.json', 'w') as outfile:
    json.dump(data, outfile)

In [11]:
# Network metrics

with open('data/social_scientists_network.json', 'r') as infile:
    data = json.load(infile)
    G = nx.node_link_graph(data)

# Total number of nodes
number_of_nodes = G.number_of_nodes()

# Total number of links
number_of_links = G.number_of_edges()

# Network density
network_density = nx.density(G)

# Determine if network is fully connected
is_connected = nx.is_connected(G)

# Determine the number of connected components
number_of_connected_components = nx.number_connected_components(G)

# Determine the number of isolated nodes
isolated_n = list(nx.isolates(G))
number_of_isolated_nodes = len(isolated_n)

print(f"Total number of nodes: {number_of_nodes}")
print(f"Total number of links: {number_of_links}")
print(f"Network density: {network_density}")
print(f"Is the network fully connected: {is_connected}")
print(f"Number of connected components: {number_of_connected_components}")
print(f"Number of isolated nodes: {number_of_isolated_nodes}")

Total number of nodes: 13032
Total number of links: 40418
Network density: 0.0004760099152519098
Is the network fully connected: False
Number of connected components: 253
Number of isolated nodes: 0


##### What is the total number of nodes (authors) and links (collaborations) in the network?

The total number of nodes (authors) is 13032.
The total number of links (collaborations) is 40418.

#### Calculate the network's density (the ratio of actual links to the maximum possible number of links). Would you say that the network is sparse? Justify your answer.

The density of the network was calculated ot be 0.00048. This is the ratio of actual links to the maximum possible number of links. For a network of 13032 nodes, this density is extremely low, indicating that the actual number of connections is much smaller than the total possible connections. Given this, we can say that the network is sparse and that most nodes are not directly connected to each other.

#### Is the network fully connected (i.e., is there a direct or indirect path between every pair of nodes within the network), or is it disconnected?

The network is not fully connected. This means that there is not a direct or indirect path between every pair of nodes in the network. Hence, it is disconnected. 

#### If the network is disconnected, how many connected components does it have? A connected component is defined as a subset of nodes within the network where a path exists between any pair of nodes in that subset.

The network has 253 connected components, meaning that there are 253 subsets of nodes within the network where each subset has a path between any pair of nodes within the subset, but no paths connect these subsets to each other. 

#### How many isolated nodes are there in your network? An isolated node is defined as a node with no connections to any other node in the network.

The network has no isolated nodes, meaning that there are no nodes with no connections to any other node in the network. 

##### Discuss the results above on network density, and connectivity. Are your findings in line with what you expected? Why? (answer in max 150 words)

The network density was calculated to be 0.00048, indicating a sparse network with a low ratio of actual links to possible links. This finding is expected in large academic collaboration networks, where not all researchers are directly connected due to their specific specializations. The network's disconnectivity, with 253 connected components and no isolated nodes, shows that the network consists of clusters representing communities within the network. These clusters likely represent subfields or groups within the field of Computational Social Science, where collaboration occurs more frequently within groups than between them. Given that there are no isolated nodes despite the network being disconnected, this suggests that each researcher has at least one collaboration. These findings align with typical expectations for academic networks, which typically consist of connected subgroups rather than one densely connected group.  

In [12]:
# Degree Analysis

import numpy as np

node_degrees = list(dict(G.degree()).values())

avg_node_degree = np.mean(node_degrees)
median_node_degree = np.median(node_degrees)
mode_node_degree = max(node_degrees, key=node_degrees.count)
min_node_degree = min(node_degrees)
max_node_degree = max(node_degrees)

print(f"Average node degree: {round(avg_node_degree, 2)}")
print(f"Median node degree: {median_node_degree}")
print(f"Modal node degree: {mode_node_degree}")
print(f"Minimum node degree: {min_node_degree}")
print(f"Maximum node degree: {max_node_degree}")
print("\n")

node_strengths = list(dict(nx.degree(G, weight='weight')).values())
avg_node_strength = np.mean(node_strengths)
median_node_strength = np.median(node_strengths)
mode_node_strength = max(node_strengths, key=node_strengths.count)
min_node_strength = min(node_strengths)
max_node_strength = max(node_strengths)

print(f"Average node strength: {round(avg_node_strength, 2)}")
print(f"Median node strength: {median_node_strength}")
print(f"Modal node strength: {mode_node_strength}")
print(f"Minimum node strength: {min_node_strength}")
print(f"Maximum node strength: {max_node_strength}")


Average node degree: 6.2
Median node degree: 5.0
Modal node degree: 3
Minimum node degree: 1
Maximum node degree: 110


Average node strength: 15.45
Median node strength: 6.0
Modal node strength: 4
Minimum node strength: 1
Maximum node strength: 1256


#### What do these metrics tell us about the network? (answer in max 150 words)

These metrics give insight into the network's structural characteristics and collaboration patterns. The average node degree at 6.20 implies each researcher collaborates with approximately 6 others, indicating notable collaboration levels. However, the median node degree of 5 and the modal degree of 3 shows many network researchers have fewer collaborations, with the most common collaboration count per researcher being 3. This difference between average and median/modal values suggests a skewed distribution, where a few researchers are significantly more connected.

Node strength metrics measure connection intensity, like repeated collaborations. The average strength, at 15.45, suggests that the intensity of collaboration per researcher is higher, factoring in collaboration frequency. The highest node strength is 1256, and the maximum node degree is 110, indicating some pairs collaborate exceptionally often, reflecting a concentrated collaboration pattern within a subset of researchers, while the majority have fewer partnerships.


In [13]:
# Top Authors

node_degrees = dict(G.degree())
top_5_authors = sorted(node_degrees.items(), key=lambda x: x[1], reverse=True)[:5]

for author, degree in top_5_authors:
    print(f"Author with id: {author} had degree: {degree}")




Author with id: https://openalex.org/A5075080019 had degree: 110
Author with id: https://openalex.org/A5065243448 had degree: 103
Author with id: https://openalex.org/A5040523461 had degree: 93
Author with id: https://openalex.org/A5044944954 had degree: 90
Author with id: https://openalex.org/A5009631424 had degree: 83


#### What role do these node play in the network?

These nodes are the key contributors within the network. Their high degrees suggest that they have collaborated with many other researchers and the nodes likely occupy a central position in their connected components. 

#### Research these authors online. What areas do they specialize in? Do you think that their work aligns with the themes of Computational Social Science? If not, what could be possible reasons? (answer in max 150 words)

The five authors are Qin Li (specializing in chemical engineering), Qin Wang (specializing in machine learning), Xinyi Wang (specializing in urban planning), Lyle H. Ungar (specializing in machine learning), and W. John Edmunds (specializing in epidemiology).


Given that Xinyi Wang specializes in urban planning and behavior analysis, her work aligns well with the themes of Computational Social Science. The other researchers have diverse backgrounds, like chemical engineering, machine learning, and epidemiology, which do not align as clearly with the themes of Computational Social Science. A possible reason for their high degrees in the network is that their work may intersect with Computational Social Science in areas where computational methods are applied to social systems analysis, predictive modelling, and understanding human behavior within their respective domains. This may have lead to these authors having many collaborations within the field, thus being a central part of the network. 
